In [ ]:
import pandas as pd
import numpy as np

## Worklist prioritization: Emergency Setting

In [ ]:
## First, read in the file of the current worklist with the probabilities that your two algorithms have
## generated for the two types of findings you're most concerned with:

worklist = pd.read_csv('probabilities.csv')

In [ ]:
worklist.head()

Here, I'm just creating a new column to address the first question in the exercise, showing that every image taking 6 minutes to read will be read in the order that they are presented in this list.

In [ ]:
worklist['time_to_read'] = np.arange(6, 6*(len(worklist)+1),6)

In [ ]:
worklist.head()

Now, for each image, I want to see if brain bleed or aortic dissection are likely. I'll create a new column showing the max probability between the two of them

In [ ]:
worklist['max_prob'] = worklist[["Brain_bleed_probability", "Aortic_dissection_probability"]].max(axis=1)

In [ ]:
worklist.head()

Great, now I want to re-order my worklist based on probabilities of critical findings:

In [ ]:
worklist_prioritized = worklist.sort_values(by=['max_prob'],ascending=False)

In [ ]:
worklist_prioritized.head()

In [ ]:
worklist_prioritized['time_to_read_prioritized'] = np.arange(6, 6*(len(worklist)+1),6)

In [ ]:
worklist_prioritized['time_delta'] = worklist_prioritized['time_to_read'] - worklist_prioritized['time_to_read_prioritized']

In [ ]:
worklist_prioritized.head()

Now, I want to find places where my algorithm saved at least 30 minutes for brain bleeds:

In [ ]:
worklist_prioritized[((worklist_prioritized.time_delta>30)&(worklist_prioritized.Image_Type=='head_ct'))]

Looks like there are 14 head CTs that were read more than 30 minutes faster than their original order. All but the last three had a probability of brain bleed < 0.4.

Do the same analysis for saving at least 15 minutes with aortic dissections: 

In [ ]:
worklist_prioritized[((worklist_prioritized.time_delta>=15)&(worklist_prioritized.Image_Type=='chest_xray'))]

In [ ]:
len(worklist_prioritized[((worklist_prioritized.time_delta>=15)&(worklist_prioritized.Image_Type=='chest_xray'))])

Looks like there are 28 chest x-rays that were read more than 15 minutes faster than their original order. All but the last nine had a probability of aortic dissection < 0.4.

Finally, I'll take a look at anywhere that my algorithm made brain bleeds or aortic dissections with a probability of 0.5 or higher be read _slower._

In [ ]:
worklist_prioritized[((worklist_prioritized.time_delta<0)&(worklist_prioritized.max_prob>=0.5))]

Looks like there were two cases where my algorithm caused an image to be read slower than the priority order it came in. Given that I had images with probabilities <0.5 that were read faster, it is definitely possible to improve my algorithm by adding some more heuristics. 